<a href="https://colab.research.google.com/github/kmouts/PMS/blob/master/AdaBoost_Ensemble_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Το Boosting (ενίσχυση) είναι μια κατηγορία αλγορίθμων μηχανικής μάθησης που περιλαμβάνει το συνδυασμό των προβλέψεων από πολλούς αδύναμους μαθητές.

Ένας αδύναμος μαθητής είναι ένα πολύ απλό μοντέλο με μια μικρή διακριτική ικανότητα στο σύνολο δεδομένων. Η Ενίσχυση ήταν μια θεωρητική ιδέα πολύ πριν μπορέσει να αναπτυχθεί ένας πρακτικός αλγόριθμος και ο αλγόριθμος **AdaBoost** (adaptive boosting - προσαρμοστική ενίσχυση) ήταν η πρώτη επιτυχημένη προσέγγιση.

Ο αλγόριθμος AdaBoost περιλαμβάνει τη χρήση πολύ μικρών (ενός επιπέδου) δέντρων αποφάσεων ως αδύναμων μαθητών που προστίθενται διαδοχικά στην ομάδα. Κάθε επόμενο μοντέλο προσπαθεί να διορθώσει τις προβλέψεις που έκανε το προηγούμενο στη σειρά μοντέλο. Αυτό επιτυγχάνεται με στάθμιση του συνόλου δεδομένων εκπαίδευσης για να δοθεί μεγαλύτερη έμφαση στα δεδομένα στα οποία τα προηγούμενα μοντέλα έκαναν σφάλματα πρόβλεψης.

Σε αυτό το εργαστήριο, θα δούμε πώς αναπτύσσουμε ομάδες AdaBoost για ταξινόμηση και παλινδρόμηση.

Με το πέρας του εργαστηρίου, θα έχουμε μάθει:

*  Ότι ensemble AdaBoost είναι μια ομάδα που δημιουργήθηκε από δέντρα αποφάσεων που προστέθηκαν διαδοχικά στο μοντέλο
*  Πώς να χρησιμοποιήσουμε το ensemble AdaBoost για ταξινόμηση και παλινδρόμηση με το scikit-learn.
*  Πώς να εξερευνήσουμε την επίδραση των υπερπαραμέτρων μοντέλου AdaBoost στην απόδοση του μοντέλου.


# Επισκόπηση εργαστηρίου
Το εργαστήριο χωρίζεται σε τέσσερα μέρη:

1.  Αλγόριθμος AdaBoost Ensemble
2.  AdaBoost Scikit-Learn API
  1.  AdaBoost για ταξινόμηση
  2.  AdaBoost για παλινδρόμηση
3.  Υπερπαράμετροι AdaBoost
  1. Αριθμός των δέντρων
  2. Αδύναμος μαθητής
  3. Ρυθμός εκμάθησης
  4. Εναλλακτικός αλγόριθμος
4. Υπερπαράμετροι AdaBoost με Grid Search 

## Αλγόριθμος AdaBoost Ensemble

Το **Boosting** αναφέρεται σε μια κατηγορία αλγορίθμων ομάδων (ensemble) μηχανικής μάθησης όπου τα μοντέλα προστίθενται διαδοχικά και τα μεταγενέστερα μοντέλα στη σειρά διορθώνουν τις προβλέψεις που έγιναν από προηγούμενα μοντέλα στην ακολουθία.

Το **AdaBoost** , συντομογραφία του "Adaptive Boosting" , είναι ένας αλγόριθμος ενισχυτικής μηχανικής εκμάθησης και ήταν μία από τις πρώτες επιτυχημένες πρακτικές προσεγγίσεις ενίσχυσης.

*Ονομάζουμε τον αλγόριθμο AdaBoost επειδή, σε αντίθεση με τους προηγούμενους αλγόριθμους, προσαρμόζεται στα σφάλματα των ασθενών υποθέσεων*
[Link](https://link.springer.com/chapter/10.1007/3-540-59119-2_166).

Το AdaBoost συνδυάζει τις προβλέψεις από σύντομα δέντρα αποφάσεων ενός επιπέδου, που ονομάζονται decision stumps (ds), αν και μπορούν να χρησιμοποιηθούν και άλλοι αλγόριθμοι. Τα πολλά αδύναμα μοντέλα ds χρησιμοποιούνται για να διορθωθούν οι προβλέψεις με πρόσθεση επιπλέον αδύναμων μοντέλων.

Ο αλγόριθμος εκπαίδευσης ξεκινά με ένα δέντρο αποφάσεων, την εύρεση εκείνων των παραδειγμάτων στα δεδομένα εκπαίδευσης που δεν ταξινομήθηκαν σωστά και την προσθήκη περισσότερου βάρους σε αυτά τα παραδείγματα. Ένα άλλο δέντρο εκπαιδεύεται στα ίδια δεδομένα, αν και τώρα σταθμίζεται από τα λάθη της προηγούμενης ταξινόμησης. Αυτή η διαδικασία επαναλαμβάνεται έως ότου προστεθεί ένας δεδομένος αριθμός δέντρων.

*Εάν ένα σημείο δεδομένων εκπαίδευσης δεν έχει ταξινομηθεί σωστά, το βάρος αυτού του σημείου εκπαίδευσης αυξάνεται (ενισχύεται). Ένας δεύτερος ταξινομητής κατασκευάζεται χρησιμοποιώντας τα νέα βάρη, τα οποία δεν είναι πλέον ίδια. Και πάλι, τα λάθος ταξινομημένα δεδομένα εκπαίδευσης αυξάνουν τα βάρη τους και η διαδικασία επαναλαμβάνεται.*
[--](https://www.intlpress.com/site/pub/pages/journals/items/sii/content/vols/0002/0003/a008/)

Ο αλγόριθμος αναπτύχθηκε για ταξινόμηση και περιλαμβάνει το συνδυασμό των προβλέψεων που έγιναν από όλα τα δέντρα αποφάσεων στο σύνολο. Μια παρόμοια προσέγγιση αναπτύχθηκε επίσης για προβλήματα παλινδρόμησης όπου γίνονται προβλέψεις χρησιμοποιώντας τον μέσο όρο των δέντρων απόφασης. Η συνεισφορά κάθε μοντέλου στην πρόβλεψη του συνόλου σταθμίζεται με βάση την απόδοση του μοντέλου στο σύνολο δεδομένων εκπαίδευσης.

*... Ο νέος αλγόριθμος δεν χρειάζεται προηγούμενη γνώση της ακρίβειας των ασθενών υποθέσεων. Αντίθετα, προσαρμόζεται σε αυτές τις ακρίβειες και δημιουργεί μια υπόθεση σταθμισμένης πλειοψηφίας στην οποία το βάρος κάθε αδύναμης υπόθεσης είναι συνάρτηση της ακρίβειας του.*
[— ](https://link.springer.com/chapter/10.1007/3-540-59119-2_166)

Τώρα που έγινε μια περιγραφή  στον αλγόριθμο AdaBoost, ας δούμε πώς μπορούμε να εκπαιδεύσουμε τέτοια μοντέλα AdaBoost στην Python.

## AdaBoost Scikit-Learn API

Το λογισμικό μηχανικής εκμάθησης **scikit-learn** σε Python παρέχει μια εφαρμογή των συνόλων AdaBoost για μηχανική μάθηση.

Διατίθεται στις τελευταίες εκδόσεις του λογισμικού. Ας δούμε την εγκατεστημένη έκδοση:

In [ ]:
# check scikit-learn version
import sklearn
print(sklearn.__version__)

Η έκδοσή σας πρέπει να είναι ίδια ή υψηλότερη από 0.22.1. Εάν όχι, πρέπει να αναβαθμίσετε την έκδοση του scikit-learn.

Το AdaBoost παρέχεται μέσω των κλάσεων [`AdaBoostRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html) και [`AdaBoostClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html).





Και τα δύο μοντέλα λειτουργούν με τον ίδιο τρόπο και λαμβάνουν τα ίδια ορίσματα που επηρεάζουν τον τρόπο δημιουργίας των δέντρων αποφάσεων.

Υπάρχει και τυχαιότητα στην κατασκευή του μοντέλου. Αυτό σημαίνει ότι κάθε φορά που ο αλγόριθμος εκτελείται στα ίδια δεδομένα, θα παράγει ένα ελαφρώς διαφορετικό μοντέλο.

Όταν χρησιμοποιούμε αλγόριθμους που έχουν στοχαστικό αλγόριθμο εκμάθησης, είναι καλή πρακτική να γίνεται η αξιολόγησή τους με τον μέσο όρο της απόδοσής τους σε πολλαπλές εκτελέσεις ή επαναλήψεις με Cross Validation. Κατά την κατασκευή ενός τελικού μοντέλου, μπορεί να είναι επιθυμητό είτε να αυξήσουμε τον αριθμό των δέντρων έως ότου μειωθεί η διακύμανση του μοντέλου σε επαναλαμβανόμενες αξιολογήσεις, είτε να κατασκευάσουμε πολλά τελικά μοντέλα και να πάρουμε τη μέση τιμή των προβλέψεών τους.

Ας δούμε πώς να αναπτύξουμε ένα ensemble AdaBoost για ταξινόμηση και για παλινδρόμηση.

### AdaBoost για Ταξινόμηση

Αρχικά, μπορούμε να χρησιμοποιήσουμε τη συνάρτηση `make_classification()` για να δημιουργήσουμε ένα συνθετικό πρόβλημα δυαδικής ταξινόμησης με 1.000 παραδείγματα και 20 χαρακτηριστικά.

In [ ]:
# test classification dataset
from sklearn.datasets import make_classification
# define dataset
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=6)
# summarize the dataset
print(X.shape, y.shape)

Στη συνέχεια, μπορούμε να αξιολογήσουμε έναν αλγόριθμο AdaBoost σε αυτό το σύνολο δεδομένων.

Θα αξιολογήσουμε το μοντέλο χρησιμοποιώντας επαναλαμβανόμενα  stratified k-fold cross-validation , με τρεις επαναλήψεις και 10 πτυχές. Θα τυπώνει τη μέση και τυπική απόκλιση της ακρίβειας του μοντέλου σε όλες τις επαναλήψεις και τις πτυχές.

In [ ]:
# evaluate adaboost algorithm for classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
# define dataset
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=6)
# define the model
model = AdaBoostClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

*Σημείωση*: Τα αποτελέσματά σας μπορεί να διαφέρουν ανάλογα με τη στοχαστική φύση του αλγορίθμου ή τη διαδικασία αξιολόγησης ή διαφορές στην αριθμητική ακρίβεια. Μπορείτε να εκτελέσετε το παράδειγμα μερικές φορές και να συγκρίνετε το μέσο αποτέλεσμα.

Σε αυτήν την περίπτωση, μπορούμε να δούμε το ensemble AdaBoost με default υπερπαραμέτρους ότι επιτυγχάνει ακρίβεια ταξινόμησης περίπου 80% σε αυτό το σύνολο δεδομένων.

Μπορούμε επίσης να χρησιμοποιήσουμε το μοντέλο AdaBoost ως τελικό μοντέλο και να κάνουμε προβλέψεις για ταξινόμηση.

Αρχικά, εκπαιδεύουμε το ensemble AdaBoost σε όλα τα διαθέσιμα δεδομένα και, στη συνέχεια, καλούμε την συνάρτηση `predict()` για να κάνει προβλέψεις σε νέα δεδομένα.

In [ ]:
# make predictions using adaboost for classification
from sklearn.datasets import make_classification
from sklearn.ensemble import AdaBoostClassifier
# define dataset
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=6)
# define the model
model = AdaBoostClassifier()
# fit the model on the whole dataset
model.fit(X, y)
# make a single prediction
row = [[-3.47224758,1.95378146,0.04875169,-0.91592588,-3.54022468,1.96405547,-7.72564954,-2.64787168,-1.81726906,-1.67104974,2.33762043,-4.30273117,0.4839841,-1.28253034,-10.6704077,-0.7641103,-3.58493721,2.07283886,0.08385173,0.91461126]]
yhat = model.predict(row)
print('Predicted Class: %d' % yhat[0])

### AdaBoost για παλινδρόμηση
Σε αυτήν την ενότητα, θα δούμε τη χρήση του AdaBoost σε ένα πρόβλημα παλινδρόμησης.

Αρχικά, χρησιμοποιούμε τη συνάρτηση `make_regression()` για να δημιουργήσουμε δεδομένα με 1.000 παραδείγματα και 20 χαρακτηριστικά.


In [ ]:
# test regression dataset
from sklearn.datasets import make_regression
# define dataset
X, y = make_regression(n_samples=1000, n_features=20, n_informative=15, noise=0.1, random_state=6)
# summarize the dataset
print(X.shape, y.shape)

Όπως κάναμε με την προηγούμενη ενότητα, θα αξιολογήσουμε το μοντέλο με k-fold cross-validation, με τρεις επαναλήψεις και 10 πτυχές. Εκτυπώνουμε το μέσο απόλυτο σφάλμα (mean absolute error - [MAE](https://scikit-learn.org/stable/modules/model_evaluation.html#mean-absolute-error) του μοντέλου σε όλες τις επαναλήψεις και πτυχές. Ένα τέλειο μοντέλο έχει MAE 0.

In [ ]:
# evaluate adaboost ensemble for regression
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import AdaBoostRegressor
# define dataset
X, y = make_regression(n_samples=1000, n_features=20, n_informative=15, noise=0.1, random_state=6)
# define the model
model = AdaBoostRegressor()
# evaluate the model
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Όπως βλέπουμε, το  ensemble AdaBoost με default υπερπαραμέτρους επιτυγχάνει MAE περίπου 70.

Μπορούμε επίσης -όπως και πριν- να χρησιμοποιήσουμε το μοντέλο AdaBoost ως τελικό μοντέλο και να κάνουμε προβλέψεις σε παλινδρόμηση.

Εκπαιδεύουμε το ensemble AdaBoost σε όλα τα διαθέσιμα δεδομένα και, στη συνέχεια, η συνάρτηση `predict()` μπορεί να κληθεί για προβλέψεις σε νέα δεδομένα.

In [ ]:
# adaboost ensemble for making predictions for regression
from sklearn.datasets import make_regression
from sklearn.ensemble import AdaBoostRegressor
# define dataset
X, y = make_regression(n_samples=1000, n_features=20, n_informative=15, noise=0.1, random_state=6)
# define the model
model = AdaBoostRegressor()
# fit the model on the whole dataset
model.fit(X, y)
# make a single prediction
row = [[1.20871625,0.88440466,-0.9030013,-0.22687731,-0.82940077,-1.14410988,1.26554256,-0.2842871,1.43929072,0.74250241,0.34035501,0.45363034,0.1778756,-1.75252881,-1.33337384,-1.50337215,-0.45099008,0.46160133,0.58385557,-1.79936198]]
yhat = model.predict(row)
print('Prediction: %d' % yhat[0])

Τώρα που είδαμε τη χρήση του scikit-learn API για αξιολόγηση και χρήση του ensemble AdaBoost, ας δούμε τη διαμόρφωση του μοντέλου.

## Υπερπαράμετροι AdaBoost
Σε αυτήν την ενότητα, θα ρίξουμε μια πιο προσεκτική ματιά σε μερικές από τις υπερπαραμέτρους που πρέπει να ρυθμίσουμε στο ensemble AdaBoost και την επίδρασή τους στην απόδοση του μοντέλου.

### Αριθμός των δέντρων
Μία σημαντική υπερπαραμέτρος για τον αλγόριθμο AdaBoost είναι ο αριθμός των δέντρων αποφάσεων που χρησιμοποιούνται στο σύνολο.

Ας θυμηθούμε ξανά ότι κάθε δέντρο αποφάσεων που χρησιμοποιείται στο σύνολο έχει σχεδιαστεί ώστε να είναι αδύναμος μαθητής. Δηλαδή, έχει δεξιότητες έναντι των τυχαίων προβλέψεων, αλλά δεν είναι ιδιαίτερα επιδέξιος. Ως τέτοια, χρησιμοποιούνται δέντρα αποφάσεων ενός επιπέδου, που ονομάζονται decision stumps.

Ο αριθμός των δέντρων που προστίθενται στο μοντέλο πρέπει να είναι μεγάλος ώστε το μοντέλο να λειτουργεί καλά, συχνά μερικές εκατοντάδες, αν όχι χιλιάδες.

Ο αριθμός των δέντρων μπορεί να οριστεί μέσω του ορίσματος `n_estimators` και έχει προεπιλεγμένη τιμή 50.

Το παρακάτω παράδειγμα διερευνά την επίδραση του αριθμού των δέντρων με τιμές μεταξύ 10 και 5.000.


In [ ]:
# explore adaboost ensemble number of trees effect on performance
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from matplotlib import pyplot

# get the dataset
def get_dataset():
	X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=6)
	return X, y

# get a list of models to evaluate
def get_models():
	models = dict()
	# define number of trees to consider
	n_trees = [10, 50, 100, 500, 1000, 5000]
	for n in n_trees:
		models[str(n)] = AdaBoostClassifier(n_estimators=n)
	return models

# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
	# define the evaluation procedure
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	# evaluate the model and collect the results
	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
	return scores

# define dataset
X, y = get_dataset()
# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	# evaluate the model
	scores = evaluate_model(model, X, y)
	# store the results
	results.append(scores)
	names.append(name)
	# summarize the performance along the way
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

Η εκτέλεση του παραδείγματος αναφέρει πρώτα τη μέση ακρίβεια για κάθε επιλεγμένο αριθμό δέντρων απόφασης.

Βλέπουμε ότι στο συγκεκριμένο σύνολο δεδομένων η απόδοση βελτιώνεται μέχρι περίπου τα 50 δέντρα και μετά από αυτό μειώνεται. Αυτό μπορεί να αποτελεί ένδειξη υπερ-προσαρμογής (overfiting) στο σύνολο δεδομένων με την προσθήκη επιπλέον δέντρων.

### Αδύναμος μαθητής
Ως προεπιλογή, ως αδύναμος μαθητής, χρησιμοποιείται ένα δέντρο αποφάσεων με ένα επίπεδο.

Μπορούμε να κάνουμε τα μοντέλα που χρησιμοποιούνται στο σύνολο να είναι λιγότερο αδύναμα (πιο επιδέξια) αυξάνοντας το βάθος του δέντρου αποφάσεων.

Το παρακάτω παράδειγμα διερευνά την επίδραση της αύξησης του βάθους τιτ `DecisionTreeClassifier` αδύναμου μαθητή στο ensemble AdBoost.


In [ ]:
# explore adaboost ensemble tree depth effect on performance
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from matplotlib import pyplot

# get the dataset
def get_dataset():
	X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=6)
	return X, y

# get a list of models to evaluate
def get_models():
	models = dict()
	# explore depths from 1 to 10
	for i in range(1,11):
		# define base model
		base = DecisionTreeClassifier(max_depth=i)
		# define ensemble model
		models[str(i)] = AdaBoostClassifier(base_estimator=base)
	return models

# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
	# define the evaluation procedure
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	# evaluate the model and collect the results
	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
	return scores

# define dataset
X, y = get_dataset()
# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	# evaluate the model
	scores = evaluate_model(model, X, y)
	# store the results
	results.append(scores)
	names.append(name)
	# summarize the performance along the way
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

Η εκτέλεση του παραδείγματος αναφέρει πρώτα τη μέση ακρίβεια για κάθε επιλεγμένου βάθους δέντρου αδύνατου μαθητή.

Βλέπουμε εδώ ότι καθώς αυξάνεται το βάθος των δέντρων αποφάσεων, η απόδοση του συνόλου  αυξάνεται σε αυτό το σύνολο δεδομένων.

### Ρυθμός εκμάθησης (learning rate)
Το AdaBoost υποστηρίζει επίσης ένα ρυθμό εκμάθησης που ελέγχει τη συμβολή κάθε μοντέλου στην συνολική πρόβλεψη.

Αυτό γίνεται με το όρισμα `learning_rate` και από προεπιλογή ορίζεται σε 1.0 ή "πλήρης συνεισφορά". Μικρότερες ή μεγαλύτερες τιμές μπορεί να είναι κατάλληλες ανάλογα με τον αριθμό των μοντέλων που χρησιμοποιούνται στο σύνολο. Υπάρχει μια ισορροπία μεταξύ της συμβολής των μοντέλων και του αριθμού των δέντρων στο σύνολο.

Περισσότερα δέντρα μπορεί να απαιτούν μικρότερο ποσοστό εκμάθησης, λιγότερα δέντρα μπορεί να απαιτούν μεγαλύτερο ποσοστό εκμάθησης. Είναι σύνηθες να χρησιμοποιούνται τιμές μεταξύ 0 και 1 και μερικές φορές πολύ μικρές τιμές για αποφυγή υπερπροσαρμογής, όπως 0.1, 0.01 ή 0.001.

Το παρακάτω παράδειγμα διερευνά τιμές ρυθμού μάθησης μεταξύ 0.1 και 2.0 με βήμα 0.1.

In [ ]:
# explore adaboost ensemble learning rate effect on performance
from numpy import mean
from numpy import std
from numpy import arange
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from matplotlib import pyplot

# get the dataset
def get_dataset():
	X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=6)
	return X, y

# get a list of models to evaluate
def get_models():
	models = dict()
	# explore learning rates from 0.1 to 2 in 0.1 increments
	for i in arange(0.1, 2.1, 0.1):
		key = '%.3f' % i
		models[key] = AdaBoostClassifier(learning_rate=i)
	return models

# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
	# define the evaluation procedure
	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
	# evaluate the model and collect the results
	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
	return scores

# define dataset
X, y = get_dataset()
# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	# evaluate the model
	scores = evaluate_model(model, X, y)
	# store the results
	results.append(scores)
	names.append(name)
	# summarize the performance along the way
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.xticks(rotation=45)
pyplot.show()

Εδώ βλέπουμε παρόμοιες τιμές μεταξύ 0.5 και 1.0 και μείωση της απόδοσης του μοντέλου μετά από αυτό.

### Εναλλακτικός αλγόριθμος
Ο προεπιλεγμένος αλγόριθμος που χρησιμοποιείται στο ensemble είναι το δέντρο αποφάσεων, αλλά μπορούν να χρησιμοποιηθούν και άλλοι αλγόριθμοι.

Ο σκοπός είναι να χρησιμοποιηθούν πολύ απλά μοντέλα, που ονομάζονται αδύναμοι μαθητές. Επίσης, το scikit-learn απαιτεί  όλα τα μοντέλα που χρησιμοποιούνται να υποστηρίζουν σταθμισμένα δείγματα, καθώς έτσι δημιουργούνται σταθμισμένες εκδόσεις του συνόλου δεδομένων εκπαίδευσης.

Το βασικό μοντέλο μπορεί να καθορίζεται μέσω του ορίσματος `base_estimator`. Το βασικό μοντέλο πρέπει επίσης να υποστηρίζει πρόβλεψη με πιθανότητες ή βαθμολογίες που μοιάζουν με πιθανότητες σε περίπτωση ταξινόμησης. Εάν το επιλεγμένο μοντέλο δεν υποστηρίζει σταθμισμένο σύνολο δεδομένων εκπαίδευσης, θα δείτε ένα παρόμοιο μήνυμα σφάλματος:

`ValueError: KNeighborsClassifier doesn't support sample_weight.`

Παράδειγμα μοντέλου που υποστηρίζει σταθμισμένη εκπαίδευση είναι ο αλγόριθμος λογιστικής παλινδρόμησης.

Το παρακάτω παράδειγμα δείχνει έναν αλγόριθμο AdaBoost με  αδύναμο μαθητή τη [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).


In [ ]:
# evaluate adaboost algorithm with logistic regression weak learner for classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
# define dataset
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=6)
# define the model
model = AdaBoostClassifier(base_estimator=LogisticRegression())
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Σε αυτήν την περίπτωση, βλέπουμε το ensemble AdaBoost με ένα αδύναμο μοντέλο λογιστικής παλινδρόμησης να επιτυγχάνει ακρίβεια ταξινόμησης περίπου 79% σε αυτό το σύνολο εκπαίδευσης.

## Υπερπαράμετροι  AdaBoost με Grid Search
Το AdaBoost μπορεί να είναι δύσκολο να βελτιστοποιηθεί ως μοντέλο, καθώς οι υπερπαράμετροι επηρεάζουν τη συμπεριφορά του μοντέλου στα δεδομένα εκπαίδευσης και επίσης αλληλεπιδρούν μεταξύ τους.

Ως εκ τούτου, είναι καλή πρακτική να χρησιμοποιείται μια διαδικασία αναζήτησης για να βρεθεί μια διαμόρφωση των  υπερπαραμέτρων που λειτουργεί καλύτερα σε ένα δεδομένο προγνωστικό πρόβλημα. Οι πιο δημοφιλείς διαδικασίες αναζήτησης περιλαμβάνουν την τυχαία (random) αναζήτηση και την αναζήτηση πλέγματος (Grid Search).

Εδώ θα εξετάσουμε την αναζήτηση πλέγματος σε συνήθεις περιοχές τιμών σε υπερπαμέτρους AdaBoost που μπορούν να χρησιμοποιηθούν ως σημείο εκκίνησης και για τα δικά σας προβλήματα. Αυτό μπορεί να επιτευχθεί χρησιμοποιώντας την τάξη `GridSearchCV` και καθορίζοντας ένα λεξικό που αντιστοιχίζει μοντέλα ονομάτων υπερπαραμέτρων σε τιμές προς αναζήτηση.

Στη συγκεκριμένη περίπτωση, θα πραγματοποιήσουμε αναζήτηση δύο βασικών υπερπαραμέτρων για το AdaBoost: τον αριθμό των δέντρων που χρησιμοποιήθηκαν στο ensemble και το ρυθμό εκμάθησης. Θα χρησιμοποιήσουμε ένα εύρος τιμών με συνήθως καλή απόδοση σε κάθε υπερπαράμετρο.

Κάθε συνδυασμός διαμόρφωσης θα αξιολογηθεί χρησιμοποιώντας επαναλαμβανόμενη k-fold cross-validation και οι διαμορφώσεις θα συγκριθούν χρησιμοποιώντας τη μέση βαθμολογία, σε αυτήν την περίπτωση, την ακρίβεια ταξινόμησης.

Το πλήρες παράδειγμα αναζήτησης πλέγματος των βασικών υπερπαραμέτρων του αλγορίθμου AdaBoost στο συνθετικό σύνολο δεδομένων ταξινόμησης παρατίθεται παρακάτω.

In [ ]:
# example of grid searching key hyperparameters for adaboost on a classification dataset
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
# define dataset
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=6)
# define the model with default hyperparameters
model = AdaBoostClassifier()
# define the grid of values to search
grid = dict()
grid['n_estimators'] = [10, 50] # [10, 50, 100, 500]
grid['learning_rate'] = [0.0001, 0.001] # [0.0001, 0.001, 0.01, 0.1, 1.0]
# define the evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# define the grid search procedure
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy')
# execute the grid search
grid_result = grid_search.fit(X, y)
# summarize the best score and configuration
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# summarize all scores that were evaluated
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Η εκτέλεση του παραδείγματος απαιτεί αρκετό χρόνο, ανάλογα με το διαθέσιμο hardware. Στο τέλος της εκτέλεσης, αναφέρεται η διαμόρφωση που πέτυχε την καλύτερη βαθμολογία, ακολουθούμενη από τα αποτελέσματα για όλες τις άλλες διαμορφώσεις που εξετάστηκαν.

Η απόδοση μπορεί να ξεπεράσει το 81% αυξάνοντας τον αριθμό δέντρων, και τον ρυθμό εκμάθησης.


*Μια υλοποίηση από το μηδέν του  αλγόριθμου AdaBoost σε Python, όπου φαίνεται η διαδικασία εκπαίδευσης με στάθμιση των δεδομένων εκπαίδευσης, μπορεί να βρεθεί [εδώ](https://geoffruddock.com/adaboost-from-scratch-in-python/).*